In [ ]:

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)



## 1. Gerador CLEVR simplificado (25 objetos) + plot
Formato do vetor de cada objeto: [x, y, r, g, b, circle, square, cylinder, cone, triangle, size_scalar] com size em [0,1]. Classes de tamanho: small (<1/3), medium (entre), big (>2/3).


In [ ]:

SHAPES = ["circle", "square", "cylinder", "cone", "triangle"]
COLORS = ["red", "green", "blue"]
SIZE_CLASSES = ["small", "medium", "big"]
RELATIONAL_PREDICATES = ["left", "right", "below", "above", "close", "can_stack"]

MARKERS = {"circle": "o", "square": "s", "cylinder": "P", "cone": "v", "triangle": "^"}
COLOR_MAP = {"red": "red", "green": "green", "blue": "blue"}
SIZE_MARKER_SCALE = {0: 60, 1: 95, 2: 130}


def size_class_from_scalar(s):
    # small < 1/3, big > 2/3, medium otherwise
    return np.digitize(s, [1/3, 2/3])


def generate_scene(n_objects: int = 25, seed: int = 0, eps: float = 0.02):
    rng = np.random.default_rng(seed)
    coords = rng.random((n_objects, 2))
    color_idx = rng.integers(0, len(COLORS), size=n_objects)
    shape_idx = rng.integers(0, len(SHAPES), size=n_objects)
    size_scalar = rng.random(n_objects)  # continuo em [0,1]
    size_idx = size_class_from_scalar(size_scalar)

    feats = np.zeros((n_objects, 11), dtype=np.float32)
    feats[:, 0:2] = coords
    feats[np.arange(n_objects), 2 + color_idx] = 1
    feats[np.arange(n_objects), 5 + shape_idx] = 1
    feats[:, 10] = size_scalar.astype(np.float32)

    feats_t = torch.tensor(feats, device=device)
    x = feats_t[:, 0]
    y = feats_t[:, 1]
    s = feats_t[:, 10]

    left = (x[:, None] + eps < x[None, :]).float()
    right = (x[:, None] > x[None, :] + eps).float()
    below = (y[:, None] + eps < y[None, :]).float()
    above = (y[:, None] > y[None, :] + eps).float()

    dist2 = (x[:, None] - x[None, :]) ** 2 + (y[:, None] - y[None, :]) ** 2
    close = torch.exp(-2 * dist2)

    in_between = (
        ((x[None, None, :] < x[:, None, None]) & (x[:, None, None] < x[None, :, None]))
        | ((x[None, :, None] < x[:, None, None]) & (x[:, None, None] < x[None, None, :]))
    ).float()

    stable = (torch.abs(x[:, None] - x[None, :]) <= 0.1) | (torch.abs(s[:, None] - s[None, :]) <= 0.15)
    can_stack = ((1 - feats_t[:, 5 + 3][:, None]) * (1 - feats_t[:, 5 + 4][:, None]) * stable.float()).float()

    labels = {
        **{sname: torch.tensor((size_idx == i).astype(np.float32), device=device) for i, sname in enumerate(SIZE_CLASSES)},
        **{s: torch.tensor((shape_idx == i).astype(np.float32), device=device) for i, s in enumerate(SHAPES)},
        **{c: torch.tensor((color_idx == i).astype(np.float32), device=device) for i, c in enumerate(COLORS)},
        "left": left,
        "right": right,
        "below": below,
        "above": above,
        "close": close,
        "in_between": in_between,
        "can_stack": can_stack,
    }

    return {
        "features": feats_t,
        "labels": labels,
        "coords": coords,
        "color_idx": color_idx,
        "shape_idx": shape_idx,
        "size_scalar": size_scalar,
        "size_idx": size_idx,
    }


def plot_scene(scene):
    coords = scene["coords"]
    shapes = scene["shape_idx"]
    colors = scene["color_idx"]
    sizes = scene["size_idx"]

    fig, ax = plt.subplots(figsize=(6, 6))
    for i, (x, y) in enumerate(coords):
        shape_name = SHAPES[shapes[i]]
        color_name = COLORS[colors[i]]
        marker = MARKERS[shape_name]
        size = SIZE_MARKER_SCALE.get(sizes[i], 90)
        ax.scatter(x, y, c=COLOR_MAP[color_name], marker=marker, s=size, edgecolor="black", alpha=0.85)
        ax.text(x + 0.01, y + 0.01, str(i), fontsize=8)

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_title("Cena CLEVR simplificada (3 tamanhos)")
    plt.show()



## 2. Conectivos e quantificadores fuzzy (LTN)
Usamos t-norm produto, t-conorm prob-sum, implicacao de Reichenbach e quantificadores por media (Forall) e max (Exists). alanced_equiv evita desbalanceamento de classes nos fatos.


In [ ]:

not_op = lambda x: 1 - x


def and_op(*xs):
    out = xs[0]
    for x in xs[1:]:
        out = out * x
    return out.clamp(0, 1)


def or_op(*xs):
    out = xs[0]
    for x in xs[1:]:
        out = out + x - out * x
    return out.clamp(0, 1)


def impl(a, b):
    return (1 - a + a * b).clamp(0, 1)


def equiv(a, b):
    return (1 - torch.abs(a - b)).clamp(0, 1)


def forall(x, dim=None):
    return x.mean() if dim is None else x.mean(dim=dim)


def exists(x, dim=None):
    return x.max() if dim is None else x.max(dim=dim).values


def prob_or(x, dim):
    return 1 - torch.prod(1 - x, dim=dim)


def balanced_equiv(pred, target):
    pred = pred.flatten()
    target = target.flatten()
    pos = target > 0.5
    neg = ~pos
    scores = []
    if pos.any():
        scores.append(equiv(pred[pos], target[pos]).mean())
    if neg.any():
        scores.append(equiv(pred[neg], target[neg]).mean())
    return torch.stack(scores).mean() if scores else torch.tensor(1.0, device=device)



## 3. Predicados neurais (LTN em PyTorch)
MLPs simples (sigmoide) para predicados unarios e binarios.


In [ ]:

class UnaryPredicate(nn.Module):
    def __init__(self, in_dim: int):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim, 32), nn.ReLU(), nn.Linear(32, 1), nn.Sigmoid())

    def forward(self, x):
        return self.net(x).squeeze(-1)


class BinaryPredicate(nn.Module):
    def __init__(self, in_dim: int):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim, 64), nn.ReLU(), nn.Linear(64, 1), nn.Sigmoid())

    def forward(self, x):
        return self.net(x).squeeze(-1)


def build_models():
    models = {s: UnaryPredicate(11).to(device) for s in SHAPES}
    models.update({c: UnaryPredicate(11).to(device) for c in COLORS})
    models.update({sname: UnaryPredicate(11).to(device) for sname in SIZE_CLASSES})
    for rel in RELATIONAL_PREDICATES:
        models[rel] = BinaryPredicate(22).to(device)
    return models



## 4. Forward de predicados, formulas (axiomas) e consultas da Tarefa 4
Inclui: unicidade/cobertura de forma, 3 tamanhos (small/medium/big), regras de left/right, below/above, closeTo, inBetween, lastOnTheLeft/right, canStack, restricao trios proximos = mesmo tamanho, opcionais e consultas compostas.


In [ ]:

DEFAULT_WEIGHTS = {
    "fact_left": 4.0,
    "fact_right": 4.0,
    "fact_below": 4.0,
    "fact_above": 4.0,
    "fact_close": 2.0,
    "fact_can_stack": 2.0,
    "fact_unary": 1.0,
    "left_irreflexive": 2.0,
    "left_asym": 2.0,
    "left_inverse": 2.0,
    "left_trans": 3.0,
    "below_inverse": 2.0,
    "below_trans": 3.0,
    "in_between_rule": 3.0,
    "tri_close_same_size": 2.0,
    "square_right_circle": 1.5,
    "exists_left_of_all_squares": 1.5,
}


def forward_preds(models, scene):
    feat = scene["features"]
    n = feat.shape[0]
    preds = {k: m(feat) for k, m in models.items() if k not in RELATIONAL_PREDICATES}

    pair_feat = torch.cat(
        [feat.unsqueeze(1).expand(n, n, -1), feat.unsqueeze(0).expand(n, n, -1)], dim=-1
    ).reshape(n * n, -1)
    for rel in RELATIONAL_PREDICATES:
        preds[rel] = models[rel](pair_feat).reshape(n, n)

    L = preds["left"]
    R = preds["right"]
    preds["in_between_pred"] = or_op(L.T.unsqueeze(2) * R.T.unsqueeze(1), L.T.unsqueeze(1) * R.T.unsqueeze(2))

    # tamanho continuo soft (0 small, 0.5 medium, 1 big)
    preds["size_soft"] = 0.0 * preds["small"] + 0.5 * preds["medium"] + 1.0 * preds["big"]
    preds["same_size"] = 1 - torch.abs(preds["size_soft"].unsqueeze(1) - preds["size_soft"].unsqueeze(0))
    return preds


def compute_formulas(models, scene):
    labels = scene["labels"]
    preds = forward_preds(models, scene)
    forms = {}
    weights = {}

    # Fatos unarios (formas, cores, tamanhos)
    for name in SHAPES + COLORS + SIZE_CLASSES:
        forms[f"fact_{name}"] = balanced_equiv(preds[name], labels[name])
        weights[f"fact_{name}"] = DEFAULT_WEIGHTS["fact_unary"]

    # Fatos binarios
    for rel in RELATIONAL_PREDICATES:
        forms[f"fact_{rel}"] = balanced_equiv(preds[rel], labels[rel])
        key = f"fact_{rel}"
        weights[key] = DEFAULT_WEIGHTS.get(key, DEFAULT_WEIGHTS["fact_close"] if rel == "close" else DEFAULT_WEIGHTS["fact_can_stack"])

    # Unicidade e cobertura de forma
    spreds = torch.stack([preds[s] for s in SHAPES], dim=-1)
    prod_pairs = torch.matmul(spreds.unsqueeze(2), spreds.unsqueeze(1))
    mask = 1 - torch.eye(len(SHAPES), device=device)
    clash = (prod_pairs * mask).max(dim=-1).values
    forms["unique_shape"] = forall(not_op(clash))
    weights["unique_shape"] = 1.0
    forms["coverage_shape"] = forall(prob_or(spreds, dim=-1))
    weights["coverage_shape"] = 1.0

    # Exclusividade e cobertura de tamanho (small/medium/big)
    tpreds = torch.stack([preds[s] for s in SIZE_CLASSES], dim=-1)
    tprod_pairs = torch.matmul(tpreds.unsqueeze(2), tpreds.unsqueeze(1))
    tmask = 1 - torch.eye(len(SIZE_CLASSES), device=device)
    tclash = (tprod_pairs * tmask).max(dim=-1).values
    forms["size_exclusive"] = forall(not_op(tclash))
    weights["size_exclusive"] = 1.0
    forms["size_coverage"] = forall(prob_or(tpreds, dim=-1))
    weights["size_coverage"] = 1.0

    # Regras de Left/Right
    L = preds["left"]
    R = preds["right"]
    forms["left_irreflexive"] = forall(not_op(torch.diag(L)))
    weights["left_irreflexive"] = DEFAULT_WEIGHTS["left_irreflexive"]
    forms["left_asym"] = forall(impl(L, not_op(L.T)))
    weights["left_asym"] = DEFAULT_WEIGHTS["left_asym"]
    forms["left_inverse"] = forall(equiv(L, R.T))
    weights["left_inverse"] = DEFAULT_WEIGHTS["left_inverse"]
    lhs_trans = L.unsqueeze(2) * L.unsqueeze(0)
    forms["left_trans"] = forall(impl(lhs_trans, L.unsqueeze(1)))
    weights["left_trans"] = DEFAULT_WEIGHTS["left_trans"]

    # Regras abaixo/acima
    B = preds["below"]
    A = preds["above"]
    forms["below_inverse"] = forall(equiv(B, A.T))
    weights["below_inverse"] = DEFAULT_WEIGHTS["below_inverse"]
    lhs_btrans = B.unsqueeze(2) * B.unsqueeze(0)
    forms["below_trans"] = forall(impl(lhs_btrans, B.unsqueeze(1)))
    weights["below_trans"] = DEFAULT_WEIGHTS["below_trans"]

    # inBetween (definido por left/right)
    forms["in_between_rule"] = forall(equiv(preds["in_between_pred"], labels["in_between"]))
    weights["in_between_rule"] = DEFAULT_WEIGHTS["in_between_rule"]

    # Triangulos proximos => mesmo tamanho (usando size_soft)
    tri_mask = preds["triangle"].unsqueeze(1) * preds["triangle"].unsqueeze(0)
    forms["tri_close_same_size"] = forall(impl(tri_mask * preds["close"], preds["same_size"]))
    weights["tri_close_same_size"] = DEFAULT_WEIGHTS["tri_close_same_size"]

    # Opcional: quadrados a direita de circulos
    forms["square_right_circle"] = forall(impl(preds["square"].unsqueeze(1) * preds["circle"].unsqueeze(0), R))
    weights["square_right_circle"] = DEFAULT_WEIGHTS["square_right_circle"]

    # Opcional: existe alguem a esquerda de todos os quadrados
    per_x = forall(impl(preds["square"].unsqueeze(0), L), dim=1)
    forms["exists_left_of_all_squares"] = exists(per_x)
    weights["exists_left_of_all_squares"] = DEFAULT_WEIGHTS["exists_left_of_all_squares"]

    sat = sum(weights[k] * v for k, v in forms.items()) / sum(weights.values())
    return sat, forms, preds


def composed_queries(preds):
    small = preds["small"]
    cylinder = preds["cylinder"]
    square = preds["square"]
    left = preds["left"]
    below = preds["below"]
    cone = preds["cone"]
    green = preds["green"]
    in_between_pred = preds["in_between_pred"]

    below_target = exists(and_op(cylinder.unsqueeze(0), below), dim=1)
    left_target = exists(and_op(square.unsqueeze(0), left), dim=1)
    q1 = exists(and_op(small, below_target, left_target))

    cone_green = cone * green
    q2 = exists(cone_green.unsqueeze(1).unsqueeze(2) * in_between_pred)

    return {
        "q1_small_below_left": q1,
        "q2_green_cone_between": q2,
    }



## 5. Treino, metricas e explicacoes (questao extra)
Treinamos maximizando satAgg ponderado. Metricas binarias (acc, precisao, recall, F1) por predicado e satisfacao de cada formula. Explicacoes textuais para Q1/Q2.


In [ ]:

THRESH = 0.5  # limiar padrao para metricas/explicacoes


def train_scene(scene, epochs: int = 250, lr: float = 0.01):
    models = build_models()
    opt = torch.optim.Adam([p for m in models.values() for p in m.parameters()], lr=lr)
    history = []
    final_forms = None
    final_preds = None

    for ep in range(epochs):
        opt.zero_grad()
        sat, forms, preds = compute_formulas(models, scene)
        loss = 1 - sat
        loss.backward()
        opt.step()

        history.append(sat.item())
        final_forms, final_preds = forms, preds

    return models, history, final_forms, final_preds


def _metrics_one(name, pred, target, threshold=THRESH):
    pred_b = (pred.detach().cpu().numpy().flatten() >= threshold).astype(int)
    tgt_b = (target.detach().cpu().numpy().flatten() >= threshold).astype(int)
    acc = (pred_b == tgt_b).mean()
    prec, rec, f1, _ = precision_recall_fscore_support(tgt_b, pred_b, average="binary", zero_division=0)
    return {"pred": name, "acc": acc, "prec": prec, "recall": rec, "f1": f1}


def compute_metrics(preds, labels, threshold=THRESH):
    rows = []
    for name in SHAPES + COLORS + SIZE_CLASSES:
        rows.append(_metrics_one(name, preds[name], labels[name], threshold))
    for rel in RELATIONAL_PREDICATES:
        rows.append(_metrics_one(rel, preds[rel], labels[rel], threshold))
    rows.append(_metrics_one("in_between", preds["in_between_pred"], labels["in_between"], threshold))
    return pd.DataFrame(rows)


def explain_queries(scene, preds, threshold=0.6):
    coords = scene["coords"]
    small = preds["small"]
    cylinder = preds["cylinder"]
    square = preds["square"]
    left = preds["left"]
    below = preds["below"]
    cone = preds["cone"]
    green = preds["green"]
    in_between_pred = preds["in_between_pred"]

    below_target = exists(and_op(cylinder.unsqueeze(0), below), dim=1)
    left_target = exists(and_op(square.unsqueeze(0), left), dim=1)
    q1_strength = small * below_target * left_target
    idxs = (q1_strength >= threshold).nonzero().flatten().tolist()
    print(f"Q1 (pequeno, abaixo de cilindro, a esquerda de quadrado) >= {threshold}: {idxs}")
    for idx in idxs[:5]:
        print(f"- obj {idx} coords={coords[idx]} forca={float(q1_strength[idx]):.3f}")

    cone_green = cone * green
    triples = cone_green.unsqueeze(1).unsqueeze(2) * in_between_pred
    best = torch.argmax(triples).item()
    n = in_between_pred.shape[0]
    x_idx = best // (n * n)
    rem = best % (n * n)
    y_idx = rem // n
    z_idx = rem % n
    print("Q2 (cone verde entre dois objetos) melhor tripla:")
    print(f"- x={x_idx} (cone*verde={float(cone_green[x_idx]):.3f}), y={y_idx}, z={z_idx}, forca={float(triples[x_idx, y_idx, z_idx]):.3f}")



## 6. Execucao exemplo unico (seed=0)
Gera cena, plota, treina, mostra satAgg final, metricas, consultas e explicacoes.


In [ ]:

scene = generate_scene(n_objects=25, seed=0)
plot_scene(scene)

models, history, forms, preds = train_scene(scene, epochs=220, lr=0.01)
print(f"satAgg final: {history[-1]:.4f}")

metrics_df = compute_metrics(preds, scene["labels"], threshold=THRESH)
display(metrics_df)

queries = composed_queries(preds)
print({k: float(v) for k, v in queries.items()})
explain_queries(scene, preds, threshold=0.6)

formula_table = pd.DataFrame([{ "formula": k, "satisfacao": float(v) } for k, v in forms.items()])
display(formula_table)



## 7. Experimentos 5x (seeds diferentes)
Repete geracao + treino para 5 seeds e consolida satAgg e metricas agregadas.


In [ ]:

results = []
for seed in range(5):
    scene_i = generate_scene(n_objects=25, seed=seed)
    _, hist_i, forms_i, preds_i = train_scene(scene_i, epochs=180, lr=0.01)
    metrics_i = compute_metrics(preds_i, scene_i["labels"], threshold=THRESH)
    results.append({
        "seed": seed,
        "satAgg": hist_i[-1],
        "acc_macro": metrics_i["acc"].mean(),
        "prec_macro": metrics_i["prec"].mean(),
        "recall_macro": metrics_i["recall"].mean(),
        "f1_macro": metrics_i["f1"].mean(),
        "q1": float(composed_queries(preds_i)["q1_small_below_left"]),
        "q2": float(composed_queries(preds_i)["q2_green_cone_between"]),
        "tri_close_same_size": float(forms_i["tri_close_same_size"]),
    })

results_df = pd.DataFrame(results)
display(results_df)



## Avaliacao da imagem do grupo
Use esta celula para validar o modelo na imagem que o professor avaliara. Ajuste objects_manual com os objetos (grid 0-20) e execute.


In [ ]:

import torch

# One-hot helpers
shape_to_vec = {
    'circle': [1,0,0,0,0],
    'square': [0,1,0,0,0],
    'cylinder': [0,0,1,0,0],
    'cone': [0,0,0,1,0],
    'triangle': [0,0,0,0,1],
}
color_to_vec = {
    'red':   [1,0,0],
    'green': [0,1,0],
    'blue':  [0,0,1],
}

def size_class_from_scalar(s):
    # small < 1/3, big > 2/3, medium otherwise
    return 0 if s <= 1/3 else (2 if s > 2/3 else 1)

def build_scene_from_objects(objs, grid_size=20.0, eps=0.02):
    rows = []
    size_idx = []
    shapes_idx = []
    colors_idx = []
    for _, x, y, color, shape, s_scalar in objs:
        xv = x / grid_size
        yv = y / grid_size
        r,g,b = color_to_vec[color]
        shv = shape_to_vec[shape]
        rows.append([xv, yv, r, g, b] + shv + [s_scalar])
        size_idx.append(size_class_from_scalar(s_scalar))
        shapes_idx.append(list(shape_to_vec.keys()).index(shape))
        colors_idx.append(list(color_to_vec.keys()).index(color))

    feats = torch.tensor(rows, dtype=torch.float32, device=device)
    x = feats[:,0]; y = feats[:,1]; s = feats[:,10]

    left  = (x[:,None] + eps < x[None,:]).float()
    right = (x[:,None] > x[None,:] + eps).float()
    below = (y[:,None] + eps < y[None,:]).float()
    above = (y[:,None] > y[None,:] + eps).float()
    dist2 = (x[:,None]-x[None,:])**2 + (y[:,None]-y[None,:])**2
    close = torch.exp(-2*dist2)
    in_between = (((x[None,None,:] < x[:,None,None]) & (x[:,None,None] < x[None,:,None])) | ((x[None,:,None] < x[:,None,None]) & (x[:,None,None] < x[None,None,:]))).float()
    stable = (torch.abs(x[:,None]-x[None,:])<=0.1) | (torch.abs(s[:,None]-s[None,:])<=0.15)
    can_stack = ((1 - feats[:,5+3][:,None]) * (1 - feats[:,5+4][:,None]) * stable.float()).float()

    labels = {
        'small': torch.tensor([1 if size_class_from_scalar(ss)==0 else 0 for *_, ss in objs], device=device, dtype=torch.float32),
        'medium': torch.tensor([1 if size_class_from_scalar(ss)==1 else 0 for *_, ss in objs], device=device, dtype=torch.float32),
        'big': torch.tensor([1 if size_class_from_scalar(ss)==2 else 0 for *_, ss in objs], device=device, dtype=torch.float32),
    }
    for sname in SHAPES:
        labels[sname] = torch.tensor([1 if shape==sname else 0 for *_, shape, _ in objs], device=device, dtype=torch.float32)
    for cname in COLORS:
        labels[cname] = torch.tensor([1 if color==cname else 0 for *_, color, _, _ in objs], device=device, dtype=torch.float32)

    labels.update({'left': left, 'right': right, 'below': below, 'above': above,
                   'close': close, 'in_between': in_between, 'can_stack': can_stack})

    return {
        'features': feats,
        'labels': labels,
        'coords': feats[:,0:2].cpu().numpy(),
        'shape_idx': shapes_idx,
        'color_idx': colors_idx,
        'size_idx': size_idx,
    }

# Preencha com os objetos da imagem (grid 0-20). Exemplo inicial aproximado:
objects_manual = [
    ('A1' ,  2 ,  2 , 'green','square'   , 0.85),
    ('A14',  5 ,  1 , 'red'  ,'square'   , 0.20),
    ('A0' ,  5 ,  4 , 'green','triangle' , 0.50),
    ('A13',  8 ,  4 , 'green','square'   , 0.40),
    ('A11', 10 ,  5 , 'green','cylinder' , 0.35),
    ('A6' ,  8 ,  5 , 'red'  ,'triangle' , 0.30),
    ('A4' ,  8 ,  9 , 'blue' ,'triangle' , 0.30),
    ('A3' ,  6 ,  9 , 'blue' ,'cylinder' , 0.35),
    ('A2' ,  4 , 10 , 'blue' ,'triangle' , 0.85),
    ('A5' ,  6 , 12 , 'red'  ,'triangle' , 0.60),
    ('A8' , 10 , 10 , 'blue' ,'circle'   , 0.30),
    ('A9' , 12 ,  9 , 'red'  ,'circle'   , 0.55),
    ('A12', 12 , 11.5,'red'  ,'circle'   , 0.55),
    ('A10', 10 , 14 , 'green','cylinder' , 0.85),
    ('A15',  4 , 16 , 'green','cone'     , 0.60),
    ('A16',  8 , 16 , 'blue' ,'cone'     , 0.60),
    ('A18', 10 , 17 , 'green','cone'     , 0.30),
    ('A19',  6 , 18 , 'blue' ,'cylinder' , 0.55),
    ('A20', 10 , 18 , 'red'  ,'cone'     , 0.30),
    ('A21', 13 , 18 , 'red'  ,'cone'     , 0.30),
    ('A22', 18 , 18 , 'red'  ,'cylinder' , 0.55),
    ('A7' , 16 , 12 , 'green','circle'   , 0.85),
    ('A17', 15 ,  8 , 'blue' ,'square'   , 0.60),
    ('A23', 16 , 10 , 'blue' ,'circle'   , 0.30),
    ('A24', 15 ,  5 , 'red'  ,'square'   , 0.60),
]

scene_img = build_scene_from_objects(objects_manual)

# Plot, treinar e avaliar
plot_scene({'coords': scene_img['coords'],
            'shape_idx': scene_img['shape_idx'],
            'color_idx': scene_img['color_idx'],
            'size_idx': scene_img['size_idx']})

models, history, forms, preds = train_scene(scene_img, epochs=220, lr=0.01)
print(f"satAgg final: {history[-1]:.4f}")
metrics_df = compute_metrics(preds, scene_img['labels'], threshold=THRESH)
display(metrics_df)

queries = composed_queries(preds)
print({k: float(v) for k, v in queries.items()})
explain_queries(scene_img, preds, threshold=0.6)

formula_table = pd.DataFrame([{ 'formula': k, 'satisfacao': float(v) } for k, v in forms.items()])
display(formula_table)
